# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/dm"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dm" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/itba2025-8d0a/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


ln: failed to create symbolic link '/content/buckets/b1/dm': File exists


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [6]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Dec 18 02:10:17 2025"

In [7]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,727412,38.9,1454584,77.7,1454584,77.7
Vcells,1337058,10.3,8388608,64.0,1975161,15.1


In [8]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [9]:
PARAM <- list()
PARAM$semilla_primigenia <- 100103

PARAM$experimento <- paste0(90102, "-", PARAM$semilla_primigenia)
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [10]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [11]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [13]:
if( !require("mice")) install.packages("mice", repos = "http://cran.us.r-project.org")
require("mice")

Loading required package: mice


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind




In [14]:
# Escrito por alumnos de  Universidad Austral  Rosario

Corregir_MICE <- function(pcampo, pmeses) {

  meth <- rep("", ncol(dataset))
  names(meth) <- colnames(dataset)
  meth[names(meth) == pcampo] <- "sample"

  # llamada a mice  !
  imputacion <- mice(dataset,
    method = meth,
    maxit = 5,
    m = 1,
    seed = 7)

  tbl <- mice::complete(dataset)

  dataset[, paste0(pcampo) := ifelse(foto_mes %in% pmeses, tbl[, get(pcampo)], get(pcampo))]

}


In [18]:
Corregir_interpolar <- function(pcampo, pmeses) {

  dataset[
    ,
    paste0(pcampo) := {
      v1 <- shift(get(pcampo), 1, type = "lag")
      v2 <- shift(get(pcampo), 1, type = "lead")
      promedio <- rowMeans(cbind(v1, v2), na.rm = TRUE)

      fifelse(
        foto_mes %in% pmeses,
        promedio,
        get(pcampo)
      )
    },
    by = dataset_metadata$PARAM$entity_id
  ]
}


In [19]:
AsignarNA_campomeses <- function(pcampo, pmeses) {

  if( pcampo %in% colnames( dataset ) ) {

    dataset[ foto_mes %in% pmeses, paste0(pcampo) := NA ]
  }
}

In [20]:
Corregir_atributo <- function(pcampo, pmeses, pmetodo)
{
  # si el campo no existe en el dataset, Afuera !
  if( !(pcampo %in% colnames( dataset )) )
    return( 1 )

  # llamo a la funcion especializada que corresponde
  switch( pmetodo,
    "MachineLearning"     = AsignarNA_campomeses(pcampo, pmeses),
    "EstadisticaClasica"  = Corregir_interpolar(pcampo, pmeses),
    "MICE"                = Corregir_MICE(pcampo, pmeses),
  )

  return( 0 )
}

In [21]:

Corregir_Rotas <- function(dataset, pmetodo) {
  gc(verbose= FALSE)
  cat( "inicio Corregir_Rotas()\n")
  # acomodo los errores del dataset

  Corregir_atributo("active_quarter", c(202006), pmetodo) # 1
  Corregir_atributo("internet", c(202006), pmetodo) # 2

  Corregir_atributo("mrentabilidad", c(201905, 201910, 202006), pmetodo) # 3
  Corregir_atributo("mrentabilidad_annual", c(201905, 201910, 202006), pmetodo) # 4

  Corregir_atributo("mcomisiones", c(201905, 201910, 202006), pmetodo) # 5

  Corregir_atributo("mactivos_margen", c(201905, 201910, 202006), pmetodo) # 6
  Corregir_atributo("mpasivos_margen", c(201905, 201910, 202006), pmetodo) # 7

  Corregir_atributo("mcuentas_saldo", c(202006), pmetodo) # 8

  Corregir_atributo("ctarjeta_debito_transacciones", c(202006), pmetodo) # 9

  Corregir_atributo("mautoservicio", c(202006), pmetodo) # 10

  Corregir_atributo("ctarjeta_visa_transacciones", c(202006), pmetodo) # 11
  Corregir_atributo("mtarjeta_visa_consumo", c(202006), pmetodo) # 12

  Corregir_atributo("ctarjeta_master_transacciones", c(202006), pmetodo) # 13
  Corregir_atributo("mtarjeta_master_consumo", c(202006), pmetodo) # 14

  Corregir_atributo("ctarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 15
  Corregir_atributo("mttarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 16

  Corregir_atributo("ccajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 17

  Corregir_atributo("mcajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 18

  Corregir_atributo("ctarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 19

  Corregir_atributo("mtarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 20

  Corregir_atributo("ctarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 21

  Corregir_atributo("mtarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 22

  Corregir_atributo("ccomisiones_otras", c(201905, 201910, 202006), pmetodo) # 23
  Corregir_atributo("mcomisiones_otras", c(201905, 201910, 202006), pmetodo) # 24

  Corregir_atributo("cextraccion_autoservicio", c(202006), pmetodo) # 25
  Corregir_atributo("mextraccion_autoservicio", c(202006), pmetodo) # 26

  Corregir_atributo("ccheques_depositados", c(202006), pmetodo) # 27
  Corregir_atributo("mcheques_depositados", c(202006), pmetodo) # 28
  Corregir_atributo("ccheques_emitidos", c(202006), pmetodo) # 29
  Corregir_atributo("mcheques_emitidos", c(202006), pmetodo) # 30
  Corregir_atributo("ccheques_depositados_rechazados", c(202006), pmetodo) # 31
  Corregir_atributo("mcheques_depositados_rechazados", c(202006), pmetodo) # 32
  Corregir_atributo("ccheques_emitidos_rechazados", c(202006), pmetodo) # 33
  Corregir_atributo("mcheques_emitidos_rechazados", c(202006), pmetodo) # 34

  Corregir_atributo("tcallcenter", c(202006), pmetodo) # 35
  Corregir_atributo("ccallcenter_transacciones", c(202006), pmetodo) # 36

  Corregir_atributo("thomebanking", c(202006), pmetodo) # 37
  Corregir_atributo("chomebanking_transacciones", c(201910, 202006), pmetodo) # 38

  Corregir_atributo("ccajas_transacciones", c(202006), pmetodo) # 39
  Corregir_atributo("ccajas_consultas", c(202006), pmetodo) # 40

  Corregir_atributo("ccajas_depositos", c(202006, 202105), pmetodo) # 41

  Corregir_atributo("ccajas_extracciones", c(202006), pmetodo) # 41
  Corregir_atributo("ccajas_otras", c(202006), pmetodo) # 43

  Corregir_atributo("catm_trx", c(202006), pmetodo) # 44
  Corregir_atributo("matm", c(202006), pmetodo) # 45
  Corregir_atributo("catm_trx_other", c(202006), pmetodo) # 46
  Corregir_atributo("matm_other", c(202006), pmetodo) # 47

  cat( "fin Corregir_rotas()\n")
}


In [23]:
# resuelvo el Catastrophe Analysis

setorder( dataset, numero_de_cliente, foto_mes )

PARAM$metodo <- "MachineLearning"

if (PARAM$metodo %in% c("MachineLearning", "EstadisticaClasica", "MICE")) {
  Corregir_Rotas(dataset, PARAM$metodo)
}


inicio Corregir_Rotas()
fin Corregir_rotas()


#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [41]:

# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)



In [42]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)


In [43]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.9903031,39.04545,38.43000,2.0014088,201901
1.9174404,38.40250,39.42800,1.9503255,201902
1.8296187,41.63947,42.54210,1.8932303,201903
1.7728863,44.27474,44.35421,1.8247220,201904
1.7212488,46.09546,46.08864,1.7460278,201905
1.6776304,45.06333,44.95500,1.6871348,201906
1.6431248,43.98333,43.75143,1.6361679,201907
1.5814483,54.84286,54.65048,1.5927530,201908
1.4947527,61.05952,58.79000,1.5549163,201909


In [44]:

drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}

In [45]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [46]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}

In [47]:
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


In [48]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}

In [49]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}

In [50]:

drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}

In [51]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios
     

[1] "mrentabilidad"                  "mrentabilidad_annual"          
 [3] "mcomisiones"                    "mactivos_margen"               
 [5] "mpasivos_margen"                "mcuenta_corriente"             
 [7] "mcaja_ahorro"                   "mcuentas_saldo"                
 [9] "mtarjeta_visa_consumo"          "mtarjeta_master_consumo"       
[11] "mprestamos_personales"          "mpayroll"                      
[13] "Master_mpagominimo"             "Visa_mpagominimo"              
[15] "mrentabilidad_lag1"             "mrentabilidad_annual_lag1"     
[17] "mcomisiones_lag1"               "mactivos_margen_lag1"          
[19] "mpasivos_margen_lag1"           "mcuenta_corriente_lag1"        
[21] "mcaja_ahorro_lag1"              "mcuentas_saldo_lag1"           
[23] "mtarjeta_visa_consumo_lag1"     "mtarjeta_master_consumo_lag1"  
[25] "mprestamos_personales_lag1"     "mpayroll_lag1"                 
[27] "Master_mpagominimo_lag1"        "Visa_mpagominimo_lag1"         
[29] "mrentabilidad_lag2"             "mrentabilidad_annual_lag2"     
[31] "mcomisiones_lag2"               "mactivos_margen_lag2"          
[33] "mpasivos_margen_lag2"           "mcuenta_corriente_lag2"        
[35] "mcaja_ahorro_lag2"              "mcuentas_saldo_lag2"           
[37] "mtarjeta_visa_consumo_lag2"     "mtarjeta_master_consumo_lag2"  
[39] "mprestamos_personales_lag2"     "mpayroll_lag2"                 
[41] "Master_mpagominimo_lag2"        "Visa_mpagominimo_lag2"         
[43] "mrentabilidad_delta1"           "mrentabilidad_delta2"          
[45] "mrentabilidad_annual_delta1"    "mrentabilidad_annual_delta2"   
[47] "mcomisiones_delta1"             "mcomisiones_delta2"            
[49] "mactivos_margen_delta1"         "mactivos_margen_delta2"        
[51] "mpasivos_margen_delta1"         "mpasivos_margen_delta2"        
[53] "mcuenta_corriente_delta1"       "mcuenta_corriente_delta2"      
[55] "mcaja_ahorro_delta1"            "mcaja_ahorro_delta2"           
[57] "mcuentas_saldo_delta1"          "mcuentas_saldo_delta2"         
[59] "mtarjeta_visa_consumo_delta1"   "mtarjeta_visa_consumo_delta2"  
[61] "mtarjeta_master_consumo_delta1" "mtarjeta_master_consumo_delta2"
[63] "mprestamos_personales_delta1"   "mprestamos_personales_delta2"  
[65] "mpayroll_delta1"                "mpayroll_delta2"               
[67] "Master_mpagominimo_delta1"      "Master_mpagominimo_delta2"     
[69] "Visa_mpagominimo_delta1"        "Visa_mpagominimo_delta2"

In [52]:

# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$metodo <- "dolar_blue"

switch(PARAM$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolar_blue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)

inicio drift_dolar_blue()
fin drift_dolar_blue()


#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [53]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "internet_lag1"                      "cliente_edad_lag1"                 
 [35] "cliente_antiguedad_lag1"            "mrentabilidad_lag1"                
 [37] "mrentabilidad_annual_lag1"          "mcomisiones_lag1"                  
 [39] "mactivos_margen_lag1"               "mpasivos_margen_lag1"              
 [41] "cproductos_lag1"                    "mcuenta_corriente_lag1"            
 [43] "mcaja_ahorro_lag1"                  "cdescubierto_preacordado_lag1"     
 [45] "mcuentas_saldo_lag1"                "ctarjeta_visa_transacciones_lag1"  
 [47] "mtarjeta_visa_consumo_lag1"         "mtarjeta_master_consumo_lag1"      
 [49] "mprestamos_personales_lag1"         "cpayroll_trx_lag1"                 
 [51] "mpayroll_lag1"                      "ccomisiones_mantenimiento_lag1"    
 [53] "ccallcenter_transacciones_lag1"     "chomebanking_transacciones_lag1"   
 [55] "ctrx_quarter_lag1"                  "Master_status_lag1"                
 [57] "Master_fechaalta_lag1"              "Master_mpagominimo_lag1"           
 [59] "Visa_status_lag1"                   "Visa_fechaalta_lag1"               
 [61] "Visa_mpagominimo_lag1"              "internet_lag2"                     
 [63] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [65] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [67] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [69] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [71] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [73] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
 [75] "ctarjeta_visa_transacciones_lag2"   "mtarjeta_visa_consumo_lag2"        
 [77] "mtarjeta_master_consumo_lag2"       "mprestamos_personales_lag2"        
 [79] "cpayroll_trx_lag2"                  "mpayroll_lag2"                     
 [81] "ccomisiones_mantenimiento_lag2"     "ccallcenter_transacciones_lag2"    
 [83] "chomebanking_transacciones_lag2"    "ctrx_quarter_lag2"                 
 [85] "Master_status_lag2"                 "Master_fechaalta_lag2"             
 [87] "Master_mpagominimo_lag2"            "Visa_status_lag2"                  
 [89] "Visa_fechaalta_lag2"                "Visa_mpagominimo_lag2"             
 [91] "internet_delta1"                    "internet_delta2"                   
 [93] "cliente_edad_delta1"                "cliente_edad_delta2"               
 [95] "cliente_antiguedad_delta1"          "cliente_antiguedad_delta2"         
 [97] "mrentabilidad_delta1"               "mrentabilidad_delta2"              
 [99] "mrentabilidad_annual_delta1"        "mrentabilidad_annual_delta2"       
[1

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [54]:

if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")
     

Loading required package: lightgbm



In [57]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [58]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$arbolitos= 20
PARAM$hojas_por_arbol= 16
PARAM$datos_por_hoja= 100
PARAM$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$arbolitos,
    num_leaves  = PARAM$hojas_por_arbol,
    min_data_in_leaf = PARAM$datos_por_hoja,
    feature_fraction_bynode  = PARAM$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )
     

In [59]:

# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202011 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202012 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

Verificacion de los campos recien creados

In [60]:
ncol(dataset)
colnames(dataset)

[1] 469

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "internet_lag1"                      "cliente_edad_lag1"                 
 [35] "cliente_antiguedad_lag1"            "mrentabilidad_lag1"                
 [37] "mrentabilidad_annual_lag1"          "mcomisiones_lag1"                  
 [39] "mactivos_margen_lag1"               "mpasivos_margen_lag1"              
 [41] "cproductos_lag1"                    "mcuenta_corriente_lag1"            
 [43] "mcaja_ahorro_lag1"                  "cdescubierto_preacordado_lag1"     
 [45] "mcuentas_saldo_lag1"                "ctarjeta_visa_transacciones_lag1"  
 [47] "mtarjeta_visa_consumo_lag1"         "mtarjeta_master_consumo_lag1"      
 [49] "mprestamos_personales_lag1"         "cpayroll_trx_lag1"                 
 [51] "mpayroll_lag1"                      "ccomisiones_mantenimiento_lag1"    
 [53] "ccallcenter_transacciones_lag1"     "chomebanking_transacciones_lag1"   
 [55] "ctrx_quarter_lag1"                  "Master_status_lag1"                
 [57] "Master_fechaalta_lag1"              "Master_mpagominimo_lag1"           
 [59] "Visa_status_lag1"                   "Visa_fechaalta_lag1"               
 [61] "Visa_mpagominimo_lag1"              "internet_lag2"                     
 [63] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [65] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [67] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [69] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [71] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [73] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
 [75] "ctarjeta_visa_transacciones_lag2"   "mtarjeta_visa_consumo_lag2"        
 [77] "mtarjeta_master_consumo_lag2"       "mprestamos_personales_lag2"        
 [79] "cpayroll_trx_lag2"                  "mpayroll_lag2"                     
 [81] "ccomisiones_mantenimiento_lag2"     "ccallcenter_transacciones_lag2"    
 [83] "chomebanking_transacciones_lag2"    "ctrx_quarter_lag2"                 
 [85] "Master_status_lag2"                 "Master_fechaalta_lag2"             
 [87] "Master_mpagominimo_lag2"            "Visa_status_lag2"                  
 [89] "Visa_fechaalta_lag2"                "Visa_mpagominimo_lag2"             
 [91] "internet_delta1"                    "internet_delta2"                   
 [93] "cliente_edad_delta1"                "cliente_edad_delta2"               
 [95] "cliente_antiguedad_delta1"          "cliente_antiguedad_delta2"         
 [97] "mrentabilidad_delta1"               "mrentabilidad_delta2"              
 [99] "mrentabilidad_annual_delta1"        "mrentabilidad_annual_delta2"       
[1

In [61]:

# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [62]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [63]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [64]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [65]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [66]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [67]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [68]:
# un Analista Jr  debe poder animarse a hacer 100 iteraciones
PARAM
num_interations <- 100

# parametros fijos del LightGBM
PARAM
param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,

  num_iterations= 9999,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 400
)

PARAM
hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower= 8L, upper= 64L),
  makeIntegerParam("min_data_in_leaf", lower= 10L, upper= 500L),
  makeNumericParam("feature_fraction", lower= 0.3, upper= 0.9),
  makeNumericParam("learning_rate", lower= 0.005, upper= 0.1)
)

$semilla_primigenia
[1] 100103

$experimento
[1] "90102-100103"

$dataset
[1] "gerencial_competencia_2025.csv.gz"

$metodo
[1] "dolar_blue"

$arbolitos
[1] 20

$hojas_por_arbol
[1] 16

$datos_por_hoja
[1] 100

$mtry_ratio
[1] 0.2

$train
$train$clase01_valor1
[1] "BAJA+2" "BAJA+1"

$train$training
[1] 202101 202102 202103


$lgb_param
$lgb_param$num_iterations
[1] 20

$lgb_param$num_leaves
[1] 16

$lgb_param$min_data_in_leaf
[1] 100

$lgb_param$feature_fraction_bynode
[1] 0.2

$lgb_param$boosting
[1] "rf"

$lgb_param$bagging_fraction
[1] 0.6321206

$lgb_param$bagging_freq
[1] 1

$lgb_param$feature_fraction
[1] 1

$lgb_param$max_bin
[1] 31

$lgb_param$objective
[1] "binary"

$lgb_param$first_metric_only
[1] TRUE

$lgb_param$boost_from_average
[1] TRUE

$lgb_param$feature_pre_filter
[1] FALSE

$lgb_param$force_row_wise
[1] TRUE

$lgb_param$verbosity
[1] -100

$lgb_param$max_depth
[1] -1

$lgb_param$min_gain_to_split
[1] 0

$lgb_param$min_sum_hessian_in_leaf
[1] 0.001

$lgb_param$lambda_l1
[1] 0

$lgb_param$lambda_l2
[1] 0

$lgb_param$pos_bagging_fraction
[1] 1

$lgb_param$neg_bagging_fraction
[1] 1

$lgb_param$is_unbalance
[1] FALSE

$lgb_param$scale_pos_weight
[1] 1

$lgb_param$drop_rate
[1] 0.1

$lgb_param$max_drop
[1] 50

$lgb_param$skip_drop
[1] 0.5

$lgb_param$extra_trees
[1] FALSE


$trainingstrategy
$trainingstrategy$validate
[1] 202107

$trainingstrategy$training
[1] 202011 202010 202009 202008 202007 202006 202005

$trainingstrategy$training_pct
[1] 1

$trainingstrategy$positivos
[1] "BAJA+1" "BAJA+2"

$semilla_primigenia
[1] 100103

$experimento
[1] "90102-100103"

$dataset
[1] "gerencial_competencia_2025.csv.gz"

$metodo
[1] "dolar_blue"

$arbolitos
[1] 20

$hojas_por_arbol
[1] 16

$datos_por_hoja
[1] 100

$mtry_ratio
[1] 0.2

$train
$train$clase01_valor1
[1] "BAJA+2" "BAJA+1"

$train$training
[1] 202101 202102 202103


$lgb_param
$lgb_param$num_iterations
[1] 20

$lgb_param$num_leaves
[1] 16

$lgb_param$min_data_in_leaf
[1] 100

$lgb_param$feature_fraction_bynode
[1] 0.2

$lgb_param$boosting
[1] "rf"

$lgb_param$bagging_fraction
[1] 0.6321206

$lgb_param$bagging_freq
[1] 1

$lgb_param$feature_fraction
[1] 1

$lgb_param$max_bin
[1] 31

$lgb_param$objective
[1] "binary"

$lgb_param$first_metric_only
[1] TRUE

$lgb_param$boost_from_average
[1] TRUE

$lgb_param$feature_pre_filter
[1] FALSE

$lgb_param$force_row_wise
[1] TRUE

$lgb_param$verbosity
[1] -100

$lgb_param$max_depth
[1] -1

$lgb_param$min_gain_to_split
[1] 0

$lgb_param$min_sum_hessian_in_leaf
[1] 0.001

$lgb_param$lambda_l1
[1] 0

$lgb_param$lambda_l2
[1] 0

$lgb_param$pos_bagging_fraction
[1] 1

$lgb_param$neg_bagging_fraction
[1] 1

$lgb_param$is_unbalance
[1] FALSE

$lgb_param$scale_pos_weight
[1] 1

$lgb_param$drop_rate
[1] 0.1

$lgb_param$max_drop
[1] 50

$lgb_param$skip_drop
[1] 0.5

$lgb_param$extra_trees
[1] FALSE


$trainingstrategy
$trainingstrategy$validate
[1] 202107

$trainingstrategy$training
[1] 202011 202010 202009 202008 202007 202006 202005

$trainingstrategy$training_pct
[1] 1

$trainingstrategy$positivos
[1] "BAJA+1" "BAJA+2"

$semilla_primigenia
[1] 100103

$experimento
[1] "90102-100103"

$dataset
[1] "gerencial_competencia_2025.csv.gz"

$metodo
[1] "dolar_blue"

$arbolitos
[1] 20

$hojas_por_arbol
[1] 16

$datos_por_hoja
[1] 100

$mtry_ratio
[1] 0.2

$train
$train$clase01_valor1
[1] "BAJA+2" "BAJA+1"

$train$training
[1] 202101 202102 202103


$lgb_param
$lgb_param$num_iterations
[1] 20

$lgb_param$num_leaves
[1] 16

$lgb_param$min_data_in_leaf
[1] 100

$lgb_param$feature_fraction_bynode
[1] 0.2

$lgb_param$boosting
[1] "rf"

$lgb_param$bagging_fraction
[1] 0.6321206

$lgb_param$bagging_freq
[1] 1

$lgb_param$feature_fraction
[1] 1

$lgb_param$max_bin
[1] 31

$lgb_param$objective
[1] "binary"

$lgb_param$first_metric_only
[1] TRUE

$lgb_param$boost_from_average
[1] TRUE

$lgb_param$feature_pre_filter
[1] FALSE

$lgb_param$force_row_wise
[1] TRUE

$lgb_param$verbosity
[1] -100

$lgb_param$max_depth
[1] -1

$lgb_param$min_gain_to_split
[1] 0

$lgb_param$min_sum_hessian_in_leaf
[1] 0.001

$lgb_param$lambda_l1
[1] 0

$lgb_param$lambda_l2
[1] 0

$lgb_param$pos_bagging_fraction
[1] 1

$lgb_param$neg_bagging_fraction
[1] 1

$lgb_param$is_unbalance
[1] FALSE

$lgb_param$scale_pos_weight
[1] 1

$lgb_param$drop_rate
[1] 0.1

$lgb_param$max_drop
[1] 50

$lgb_param$skip_drop
[1] 0.5

$lgb_param$extra_trees
[1] FALSE


$trainingstrategy
$trainingstrategy$validate
[1] 202107

$trainingstrategy$training
[1] 202011 202010 202009 202008 202007 202006 202005

$trainingstrategy$training_pct
[1] 1

$trainingstrategy$positivos
[1] "BAJA+1" "BAJA+2"

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [69]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [70]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

ERROR: Error in makeObjectiveFunction(name, id, description, fn, has.simple.signature, : Assertion on 'par.set' failed: Must inherit from class 'ParamSet', but has class 'NULL'.


Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [62]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Fri Dec 12 01:03:25 AM 2025 AUC 0.928927264764968

Fri Dec 12 01:05:33 AM 2025 AUC 0.927382008855853

Fri Dec 12 01:09:04 AM 2025 AUC 0.928772947710616

Fri Dec 12 01:13:40 AM 2025 AUC 0.934168251930701

Fri Dec 12 01:16:12 AM 2025 AUC 0.927528911276962

Fri Dec 12 01:18:04 AM 2025 AUC 0.929757471980796

Fri Dec 12 01:20:13 AM 2025 AUC 0.925878460258724

Fri Dec 12 01:21:54 AM 2025 AUC 0.928304667279919

[mbo] 0: num_leaves=66; min_data_in_leaf=3631 : y = 0.929 : 99.8 secs : initdesign

[mbo] 0: num_leaves=178; min_data_in_leaf=4210 : y = 0.927 : 128.6 secs : initdesign

[mbo] 0: num_leaves=141; min_data_in_leaf=5195 : y = 0.929 : 211.1 secs : initdesign

[mbo] 0: num_leaves=115; min_data_in_leaf=1192 : y = 0.934 : 275.9 secs : initdesign

[mbo] 0: num_leaves=229; min_data_in_leaf=6625 : y = 0.928 : 151.4 secs : initdesign

[mbo] 0: num_leaves=26; min_data_in_leaf=14 : y = 0.93 : 112.3 secs : initdesign

[mbo] 0: num_leaves=56; min_data_

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [63]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        116              667            744


### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [64]:
PARAM$trainingstrategy$final_train <- c(202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 240562

##### Final Training Hyperparameters

In [65]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [66]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [67]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [68]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [69]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [70]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [71]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [72]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "data-mining-gerencial-2025-c"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 
Successfully submitted to Data Mining, Gerencial 2025 C 


In [73]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [74]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Dec 12 01:49:59 AM 2025"